# LLaMa-Alpaca-LoRA 파인튜닝

나만의 데이터로 ChatGPT 만들기

> 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

![](https://github.com/kairess/alpaca-lora/raw/main/result.jpg)

In [1]:
!nvidia-smi

Fri Mar  8 07:04:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Download source code and install dependencies

- https://github.com/tloen/alpaca-lora.git
- 주의!: bitsandbytes가 현재 Linux에서만 동작함
- 주의!!: peft 버그 https://github.com/tloen/alpaca-lora/issues/293

In [2]:
!git clone -q https://github.com/kairess/alpaca-lora.git
%cd alpaca-lora

/content/alpaca-lora


In [3]:
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━

## Dataset

### - 49,620개의 Instruction, Input, Response set.

- https://github.com/Beomi/KoAlpaca/blob/main/ko_alpaca_data.json
- https://huggingface.co/datasets/Bingsu/ko_alpaca_data

### - shareGPT 한국어 번역 데이터셋

이 프로젝트는 shareGPT 데이터셋 60만 대화문을 DeepL 을 통해 한국어로 번역하고 있습니다.

https://huggingface.co/datasets/junelee/sharegpt_deepl_ko

```json
[
  {
    "instruction":"직원들의 급여를 기록하는 데이터베이스를 설계하십시오.",
    "input":"",
    "output":"직원들의 급여를 새로운 데이터베이스에서 추적하려면, 다음과 같은 열이 포함된 \"Employee\" 테이블을 만들어야 합니다: \"name\", \"salary\", \"start_date\", \"end_date\"."
  },
  {
    "instruction":"아래 문장의 각 단어에 품사를 지정하십시오.",
    "input":"새끼 고양이는 종종 신나게 뛰어다니기도 합니다.",
    "output":"새끼(Noun) 고양이(Noun) 는 은 종종 Adverb 신나게 Adverb 뛰어다니기도 합니다."
  },
  ...
]
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content/alpaca-lora'

In [4]:
import json

with open('/content/alpaca-lora/custom_data.json', 'r') as f:
    custom_data = json.load(f)

custom_data[:5]

JSONDecodeError: Unterminated string starting at: line 149744 column 18 (char 29304742)

## Finetune

기본 설정으로 했을 때:
- 3h/1epoch in NVIDIA A100
- 12h/1epoch in NVIDIA T4

### Create a custom template

Default Alpaca

```json
{
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

In [ ]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an example pairing news article text with questions and answers about the article.\n"
        "아래는 뉴스기사 본문과 뉴스기사에 대한 질문 및 그에 대한 답변이 짝을 이루는 예제 입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(질문):\n{instruction}\n\n### Input(뉴스기사):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is the news article text along with an explanation of the news article.\n"
        "아래는 질문과 그에 대한 답변 입니다.\n\n"
        "Write a response that appropriately completes the question.\n질문에 대해 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(질문):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

### Hyperparameters

https://github.com/tloen/alpaca-lora/blob/0e1a5d52a460d14aea2325e43c302972badb9cdd/finetune.py#L28

In [ ]:
# import shutil
# shutil.rmtree('/content/alpaca-lora/output')

In [ ]:
!python finetune.py \
        --base_model 'baffo32/decapoda-research-llama-7B-hf' \
        --data_path '/content/drive/MyDrive/LLM project/0.프로젝트4_마지막프로젝트_240226/BERT_Training/custom_data.json' \
        --output_dir './output' \
        --num_epochs 5 \
        --learning_rate 5e-5 \
        --val_set_size 2000 \
        --batch_size 256 \
        --micro_batch_size 64 \
        --prompt_template_name 'custom'

Training Alpaca-LoRA model with params:
base_model: baffo32/decapoda-research-llama-7B-hf
data_path: /content/drive/MyDrive/LLM project/0.프로젝트4_마지막프로젝트_240226/BERT_Training/custom_data.json
output_dir: ./output
batch_size: 256
micro_batch_size: 64
num_epochs: 5
learning_rate: 5e-05
cutoff_len: 256
val_set_size: 2000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: False
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: custom

Loading checkpoint shards: 100% 33/33 [00:15<00:00,  2.13it/s]
trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199
Map: 100% 29993/29993 [01:52<00:00, 266.30 examples/s]
Map: 100% 2000/2000 [00:07<00:00, 263.14 examples/s]
2024-03-07 10:54:55.948744: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting t

In [ ]:
!mkdir /content/drive/MyDrive/LLaMa-Alpaca-LoRA
!cp -a output /content/drive/MyDrive/LLaMa-Alpaca-LoRA

## Test on Gradio

--load_8bit

In [ ]:
!python generate.py \
    --base_model 'baffo32/decapoda-research-llama-7B-hf' \
    --lora_weights '/content/alpaca-lora/output' \
    --prompt_template 'custom' \
    --share_gradio

tokenizer.model: 100% 500k/500k [00:00<00:00, 29.4MB/s]
special_tokens_map.json: 100% 2.00/2.00 [00:00<00:00, 12.1kB/s]
tokenizer_config.json: 100% 142/142 [00:00<00:00, 823kB/s]
config.json: 100% 428/428 [00:00<00:00, 2.26MB/s]
pytorch_model.bin.index.json: 100% 25.5k/25.5k [00:00<00:00, 65.2MB/s]
pytorch_model-00001-of-00033.bin:   0% 0.00/405M [00:00<?, ?B/s]
pytorch_model-00001-of-00033.bin:  10% 41.9M/405M [00:00<00:01, 334MB/s]
pytorch_model-00001-of-00033.bin:  21% 83.9M/405M [00:00<00:00, 355MB/s]
pytorch_model-00001-of-00033.bin:  31% 126M/405M [00:00<00:00, 365MB/s] 
pytorch_model-00001-of-00033.bin:  41% 168M/405M [00:00<00:00, 373MB/s]
pytorch_model-00001-of-00033.bin:  52% 210M/405M [00:00<00:00, 373MB/s]
pytorch_model-00001-of-00033.bin:  62% 252M/405M [00:00<00:00, 372MB/s]
pytorch_model-00001-of-00033.bin:  73% 294M/405M [00:00<00:00, 373MB/s]
pytorch_model-00001-of-00033.bin:  83% 336M/405M [00:00<00:00, 377MB/s]
pytorch_model-00001-of-00033.bin: 100% 405M/405M [00:01<